In [1]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace, date_format 
from pyspark.sql.types import IntegerType, DoubleType, FloatType

In [2]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatel0311"

In [3]:
df = spark.read.csv(dir_hdfs)

In [4]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)

In [5]:
df.printSchema()

root
 |-- id_folio: string (nullable = true)
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tipo_de_entrada: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- fecha_concluido: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- 0311_colonia_registro: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [6]:
print(f"Numero de Renglones: {df.count()}")
print(f"Numero de Columnas: {len(df.columns)}")
print(f"Columnas {df.columns}")

Numero de Renglones: 2323089
Numero de Columnas: 15
Columnas ['id_folio', 'fecha_solicitud', 'hora_solicitud', 'tipo_de_entrada', 'tema_solicitud', 'sexo', 'edad', 'estatus', 'fecha_concluido', 'alcaldia', 'colonia_datos', '0311_colonia_registro', 'codigo_postal_solicitud', 'latitud', 'longitud']


In [7]:
df.show(truncate=False)

+--------+---------------+--------------+---------------------+---------------------------------------+---------------+----+--------+---------------+--------+-------------+---------------------+-----------------------+-------+--------+
|id_folio|fecha_solicitud|hora_solicitud|tipo_de_entrada      |tema_solicitud                         |sexo           |edad|estatus |fecha_concluido|alcaldia|colonia_datos|0311_colonia_registro|codigo_postal_solicitud|latitud|longitud|
+--------+---------------+--------------+---------------------+---------------------------------------+---------------+----+--------+---------------+--------+-------------+---------------------+-----------------------+-------+--------+
|- 244841|2020-01-11     |08:17:58      |VICTORIA             |FUGA DE AGUA                           |NO ESPECIFICADO|NA  |ATENDIDO|2020-07-03     |NA      |NA           |NA                   |NA                     |0      |0       |
|- 265596|2020-01-28     |12:10:42      |VICTORIA       

In [8]:
datos = df.select(['fecha_solicitud' , 'hora_solicitud' , 'tema_solicitud' , 'sexo' , 'edad' , 'estatus' ,
'alcaldia', 'colonia_datos' , 'codigo_postal_solicitud' , 'latitud' , 'longitud'])

In [9]:
datos.printSchema()

root
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [10]:
datos.count()

2323089

In [11]:
datos.filter('sexo ="FEMENINO"').count()

829463

In [12]:
datos.select(['fecha_solicitud' , 'hora_solicitud' , 'tema_solicitud' , 'edad' , 'estatus' ,
'alcaldia', 'colonia_datos']).filter('sexo ="FEMENINO"').orderBy('fecha_solicitud').show()

+---------------+--------------+--------------------+----+--------+-----------------+--------------------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|edad| estatus|         alcaldia|       colonia_datos|
+---------------+--------------+--------------------+----+--------+-----------------+--------------------+
|     2019-06-02|      16:50:47|           ALUMBRADO|  29|ATENDIDO|       CUAUHTEMOC|      ROMA NORTE III|
|     2019-06-02|      23:30:08|       FALTA DE AGUA|  50|ATENDIDO|               NA|                  NA|
|     2019-06-02|      17:28:06|      MEDIO AMBIENTE|  19|ATENDIDO|               NA|                  NA|
|     2019-06-02|      14:40:34|        FUGA DE AGUA|  38|ATENDIDO|          TLALPAN|FLORESTA-PRADO-VE...|
|     2019-06-02|      17:33:25|MANTENIMIENTO VÍA...|  71|ATENDIDO|       IZTAPALAPA|          MODELO (U)|
|     2019-06-02|      15:17:44|              BACHEO|  66|ATENDIDO|         COYOACAN|  AJUSCO HUAYAMILPAS|
|     2019-06-02|      17:40:36|     

In [13]:
type(datos)

pyspark.sql.dataframe.DataFrame

In [14]:
#Transformar Columnas Cast
datos = datos.withColumn("fecha_solicitud", to_date(col("fecha_solicitud"), "yyy-MM-dd"))\
        .withColumn("hora_solicitud", date_format(col("hora_solicitud"), "HH:mm:ss"))\
        .withColumn("edad", col("edad").cast(FloatType()))\
        .withColumn("latitud", col("latitud").cast(DoubleType()))\
        .withColumn("longitud", col("longitud").cast(DoubleType()))\

datos.printSchema()
datos.show()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)

+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-----------------------+-------+--------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|           sexo|edad| estatus|alcaldia|colonia_datos|codigo_postal_solicitud|latitud|longitud|
+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-----------------------+-------+--------+
|     2020-01-11|      08:17:58|        FUGA DE AGUA|NO ESPECIFICADO|NULL|ATENDIDO| 

In [16]:
# Contar Nulos por Columna
datos.select([sum(col(c).isNull().cast("int")).alias(c) for c in datos.columns]).show()

+---------------+--------------+--------------+----+-------+-------+--------+-------------+-----------------------+-------+--------+
|fecha_solicitud|hora_solicitud|tema_solicitud|sexo|   edad|estatus|alcaldia|colonia_datos|codigo_postal_solicitud|latitud|longitud|
+---------------+--------------+--------------+----+-------+-------+--------+-------------+-----------------------+-------+--------+
|              0|             0|             0|   0|1402897|      0|       0|            0|                      0| 682509|  431500|
+---------------+--------------+--------------+----+-------+-------+--------+-------------+-----------------------+-------+--------+



In [44]:
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs="fecha_solicitud"
colhs = "hora_solicitud"

datos_tiempo = datos.select(
col("edad"), col("estatus"), col("tema_solicitud"),
year(col(colfs)).alias("anio"),
month(col(colfs)).alias("mes"),
dayofmonth(col(colfs)).alias("dia"),
weekofyear(col(colfs)).alias("semana"),
date_format(col(colfs),"MMMM").alias("nombre_mes"),
date_format(col(colfs),"EEEE").alias("nombre_dia"),
hour(col(colfs)).alias("hora"),
minute(col(colfs)).alias("minuto"),
second(col(colfs)).alias("segundo")
)

datos_tiempo.printSchema()

root
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- semana: integer (nullable = true)
 |-- nombre_mes: string (nullable = true)
 |-- nombre_dia: string (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- segundo: integer (nullable = true)



In [42]:
datos_tiempo.show(truncate = False)

+----+--------+---------------------------------------+----+---+---+------+----------+----------+----+------+-------+
|edad|estatus |tema_solicitud                         |anio|mes|dia|semana|nombre_mes|nombre_dia|hora|minuto|segundo|
+----+--------+---------------------------------------+----+---+---+------+----------+----------+----+------+-------+
|NULL|ATENDIDO|FUGA DE AGUA                           |2020|1  |11 |2     |January   |Saturday  |0   |0     |0      |
|NULL|ATENDIDO|VENTA DE ALCOHOL / DROGA               |2020|1  |28 |5     |January   |Tuesday   |0   |0     |0      |
|NULL|ATENDIDO|TRAMITES / INFO VEHICULAR              |2020|1  |10 |2     |January   |Friday    |0   |0     |0      |
|NULL|ATENDIDO|MANTENIMIENTO DRENAJE                  |2020|1  |24 |4     |January   |Friday    |0   |0     |0      |
|NULL|ATENDIDO|SOLICITUD DE VIGILANCIA                |2020|1  |4  |1     |January   |Saturday  |0   |0     |0      |
|NULL|ATENDIDO|ALERTA SISMICA                         |2

In [43]:
datos_tiempo.count()

2323089

In [45]:
#Estadísticos Descriptivos del Dataframe

datos_tiempo.describe().show()

+-------+------------------+--------------------+--------------------+------------------+------------------+-----------------+-----------------+----------+----------+-------+-------+-------+
|summary|              edad|             estatus|      tema_solicitud|              anio|               mes|              dia|           semana|nombre_mes|nombre_dia|   hora| minuto|segundo|
+-------+------------------+--------------------+--------------------+------------------+------------------+-----------------+-----------------+----------+----------+-------+-------+-------+
|  count|            920192|             2323089|             2323089|           2323089|           2323089|          2323089|          2323089|   2323089|   2323089|2323089|2323089|2323089|
|   mean| 39.27146399881764|                18.0|                NULL|2021.5103188039718|6.4875654785503265|15.85532538787795|26.43079107171529|      NULL|      NULL|    0.0|    0.0|    0.0|
| stddev|27.238747481977125|                 

In [46]:
#Estadísticos Descriptivos de algunas columnas del dataframe

columnas = ['edad', 'anio', 'mes', 'dia', 'hora', 'minuto', 'segundo', 'semana']

datos_tiempo.select(columnas).describe().show()

+-------+------------------+------------------+------------------+-----------------+-------+-------+-------+-----------------+
|summary|              edad|              anio|               mes|              dia|   hora| minuto|segundo|           semana|
+-------+------------------+------------------+------------------+-----------------+-------+-------+-------+-----------------+
|  count|            920192|           2323089|           2323089|          2323089|2323089|2323089|2323089|          2323089|
|   mean| 39.27146399881764|2021.5103188039718|6.4875654785503265|15.85532538787795|    0.0|    0.0|    0.0|26.43079107171529|
| stddev|27.238747481977125|  1.37871177376573|3.4064677225068465| 8.65088685805828|    0.0|    0.0|    0.0|14.82067661213932|
|    min|               0.0|              2019|                 1|                1|      0|      0|      0|                1|
|    max|           16090.0|              2024|                12|               31|      0|      0|      0|   

In [47]:
datos_tiempo.select(columnas).describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
edad,920192,39.27146399881764,27.238747481977125,0.0,16090.0
anio,2323089,2021.5103188039718,1.37871177376573,2019,2024
mes,2323089,6.4875654785503265,3.4064677225068465,1,12
dia,2323089,15.85532538787795,8.65088685805828,1,31
hora,2323089,0.0,0.0,0,0
minuto,2323089,0.0,0.0,0,0
segundo,2323089,0.0,0.0,0,0
semana,2323089,26.43079107171529,14.82067661213932,1,53


In [50]:
# Ver valores únicos

print(datos_tiempo.select("tema_solicitud").distinct().count())
datos_tiempo.select("tema_solicitud").distinct().show(truncate = False)

80
+--------------------------------+
|tema_solicitud                  |
+--------------------------------+
|VERIFICACIÓN ADMINISTRATIVA     |
|BACHEO                          |
|REPARACION DE EMPEDRADO         |
|BECAS                           |
|FALTA DE AGUA                   |
|MERCADOS PÚBLICOS, TIANGUIS     |
|REINSCRIPCION AL CENDI          |
|LLAVE CDMX                      |
|ASESORÍA DE TERCEROS ACREDITADOS|
|QUEJAS Y DENUNCIAS              |
|WIFI GRATIS                     |
|REGISTRO CIVIL                  |
|CONSTRUCCIONES Y OBRAS          |
|ALARMAS VECINALES               |
|SOLICITUD/SEGURO DE DESEMPLEO   |
|AVISO PREVIO DE MANIFESTACIONES |
|SOLICITUD DE VIGILANCIA         |
|FOMENTO ECONÓMICO Y EMPLEO      |
|PODA / RETIRO ARBOL             |
|INFORMACIÓN SOBRE TRÁMITES      |
+--------------------------------+
only showing top 20 rows



In [52]:
datos_tiempo.select("tema_solicitud").distinct().orderBy("tema_solicitud").show(100, truncate = False)

+----------------------------------------+
|tema_solicitud                          |
+----------------------------------------+
|AGUA POTABLE Y SERVICIOS HIDRÁULICOS    |
|ALARMAS VECINALES                       |
|ALERTA SISMICA                          |
|ALUMBRADO                               |
|ANUNCIOS                                |
|APOYO SERVICIOS FUNERARIOS              |
|ASESORIAS                               |
|ASESORÍA DE TERCEROS ACREDITADOS        |
|ASESORÍA JURÍDICA                       |
|ASISTENCIA SOCIAL                       |
|ASISTENCIA VETERINARIA                  |
|AVISO PREVIO DE MANIFESTACIONES         |
|BACHEO                                  |
|BALIZAMIENTO                            |
|BARBECHO / CHAPONEO                     |
|BECAS                                   |
|CHATARRIZACIÓN                          |
|CONSTRUCCIONES Y OBRAS                  |
|COVID                                   |
|CRÉDITO DE VIVIENDA                     |
|CULTURA Y 

In [54]:
datos_tiempo.groupBy(["tema_solicitud"]).count().orderBy("count", ascending = False).show(100, truncate = False)

+----------------------------------------+------+
|tema_solicitud                          |count |
+----------------------------------------+------+
|OTRO                                    |303078|
|ALUMBRADO                               |259590|
|PODA / RETIRO ARBOL                     |214579|
|FALTA DE AGUA                           |205785|
|FUGA DE AGUA                            |109632|
|BACHEO                                  |96571 |
|MANTENIMIENTO VÍA PÚBLICA               |95159 |
|DESAZOLVE                               |93337 |
|RETIRO CASCAJO, ESCOMBRO, AZOLVE, RAMAS |75400 |
|MANTENIMIENTO DE COLADERA / ALCANTARILLA|54717 |
|TRAMITES / INFO VEHICULAR               |46176 |
|ALERTA SISMICA                          |44182 |
|QUEJA DE TRANSPORTE PUBLICO             |42056 |
|VERIFICACIÓN ADMINISTRATIVA             |40880 |
|COVID                                   |37983 |
|RECOLECCIÓN BASURA                      |37071 |
|VEHÍCULO ABANDONADO / CHATARRIZACIÓN    |36450 |


In [58]:
datos_tiempo.groupBy(["tema_solicitud"]).count().orderBy("count", ascending = False)\
.filter("count >=90000")\
.show(100, truncate = False)

+-------------------------+------+
|tema_solicitud           |count |
+-------------------------+------+
|OTRO                     |303078|
|ALUMBRADO                |259590|
|PODA / RETIRO ARBOL      |214579|
|FALTA DE AGUA            |205785|
|FUGA DE AGUA             |109632|
|BACHEO                   |96571 |
|MANTENIMIENTO VÍA PÚBLICA|95159 |
|DESAZOLVE                |93337 |
+-------------------------+------+



In [60]:
datos_tiempo.groupBy(["anio","tema_solicitud"]).count().orderBy("tema_solicitud", ascending = True)\
.show(100, truncate = False)

+----+------------------------------------+-----+
|anio|tema_solicitud                      |count|
+----+------------------------------------+-----+
|2020|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|4487 |
|2021|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|11114|
|2019|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|1972 |
|2020|ALARMAS VECINALES                   |205  |
|2021|ALARMAS VECINALES                   |110  |
|2022|ALARMAS VECINALES                   |549  |
|2023|ALARMAS VECINALES                   |301  |
|2019|ALARMAS VECINALES                   |233  |
|2024|ALARMAS VECINALES                   |12   |
|2020|ALERTA SISMICA                      |5379 |
|2021|ALERTA SISMICA                      |25801|
|2022|ALERTA SISMICA                      |7849 |
|2023|ALERTA SISMICA                      |3008 |
|2019|ALERTA SISMICA                      |1900 |
|2024|ALERTA SISMICA                      |245  |
|2020|ALUMBRADO                           |42359|
|2021|ALUMBRADO                           |56755|


In [63]:
datos_tiempo.groupBy(["anio","tema_solicitud"]).count().orderBy(["tema_solicitud", "anio"], ascending = [True, True])\
.show(100, truncate = False)

+----+------------------------------------+-----+
|anio|tema_solicitud                      |count|
+----+------------------------------------+-----+
|2019|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|1972 |
|2020|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|4487 |
|2021|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|11114|
|2019|ALARMAS VECINALES                   |233  |
|2020|ALARMAS VECINALES                   |205  |
|2021|ALARMAS VECINALES                   |110  |
|2022|ALARMAS VECINALES                   |549  |
|2023|ALARMAS VECINALES                   |301  |
|2024|ALARMAS VECINALES                   |12   |
|2019|ALERTA SISMICA                      |1900 |
|2020|ALERTA SISMICA                      |5379 |
|2021|ALERTA SISMICA                      |25801|
|2022|ALERTA SISMICA                      |7849 |
|2023|ALERTA SISMICA                      |3008 |
|2024|ALERTA SISMICA                      |245  |
|2019|ALUMBRADO                           |37009|
|2020|ALUMBRADO                           |42359|
